In [80]:
import spacy
import pandas as pd
import random
import re
import spacy
import time
import utils.functions as f
from utils.stop_words_custom import my_stop_words as stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
nlp = spacy.load('pt_core_news_sm')
pd.set_option('display.max_colwidth', -1)

In [81]:
def clean_tweet(tweet):
    cleaned = f.toFixPunctuation(tweet)
    cleaned = f.toLower(cleaned)
    cleaned = f.changePatterns(cleaned)
    #cleaned = f.removeHashtag(cleaned)
    cleaned = f.removeLink(cleaned)
    #cleaned = f.removeMention(cleaned)
    cleaned = f.remove_punctuation(cleaned)
    cleaned = f.toFixBlankSpaces(cleaned)
    cleaned = f.remove_spaces_between_words(cleaned)
    return cleaned

In [5]:
df_positive = f.get_data('data/10_classify_tweets/classify_tweets.xlsx')
df_negative = f.get_data('data/11_classified/negative.xlsx')
df_positive2 = f.get_data('data/11_classified/positive.xlsx')

In [6]:
df_negative.shape

(16593, 1)

In [7]:
df_positive.shape

(2119, 2)

In [8]:
df_positive2.shape

(6245, 1)

In [9]:
df_positive.columns

Index(['description', 'class'], dtype='object')

In [10]:
df_positive = df_positive[df_positive["class"]=="positive"]

In [11]:
df_positive.head(1)

,description,class
3,"@DeputadoFederal Meu Deus, eu não te bloqueei ainda, me perdoe nobre excremento, eu já devia ter feito isto antes, foi mal mesmo.\n#Bolsonaro2022 \n#Bolsonaro",positive


In [12]:
df_negative.head(1)

,0
0,"A família Bolsonaro, a morte de Marielle e os milicianos presos no Rio, por Lucia Helena Issa – GGN https://t.co/MsCtOuHah0"


In [13]:
df_negative.columns = ["description"]
df_negative.head(1)

,description
0,"A família Bolsonaro, a morte de Marielle e os milicianos presos no Rio, por Lucia Helena Issa – GGN https://t.co/MsCtOuHah0"


In [14]:
df_negative["class"] = "negative"

In [15]:
df_negative.head(1)

,description,class
0,"A família Bolsonaro, a morte de Marielle e os milicianos presos no Rio, por Lucia Helena Issa – GGN https://t.co/MsCtOuHah0",negative


In [16]:
df_negative["class"].value_counts()

negative    16593
Name: class, dtype: int64

In [17]:
df_positive2.columns = ["description"]
df_positive2["class"] = "positive"

In [18]:
df_all_classified = pd.concat([df_negative, df_positive, df_positive2])
df_all_classified["class"].value_counts()

negative    16593
positive    7040 
Name: class, dtype: int64

In [19]:
df_description_cleaned = df_all_classified['description'].apply(clean_tweet)
train_X, test_X, train_y, test_y = train_test_split(df_description_cleaned, df_all_classified['class'], test_size=0.5, random_state=42, stratify=df_all_classified['class'])

In [20]:
# vectorizer = CountVectorizer(ngram_range=(1,2))
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))

In [21]:
train_X = vectorizer.fit_transform(train_X)
test_X = vectorizer.transform(test_X)

clf = MultinomialNB()
clf.fit(train_X, train_y)

print("The accuracy on the test set is %.2f. The ngram representation had %i features." % (clf.score(test_X, test_y), train_X.shape[1]))

The accuracy on the test set is 0.93. The ngram representation had 179564 features.


In [24]:
#print(vectorizer.get_feature_names())

In [ ]:
# prediction = clf.predict(vectorizer.transform(["O Bolsonaro vai se reeleger em 2022."]))
# print(prediction)

In [25]:
# prediction = clf.predict(vectorizer.transform(["O Bolsonaro matou a Marielle."]))
# print(prediction)

In [26]:
# bow_matrix = vectorizer.fit_transform(df["description"])
# print(bow_matrix.shape)

In [27]:
# bow_matrix = vectorizer.fit_transform(description_series)
# print(bow_matrix.shape)

In [28]:
df_all = f.get_data('data/07_repeated_descriptions_deleted/without_repeated_descriptions.xlsx')

In [29]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226773 entries, 834 to 20829
Data columns (total 19 columns):
pubdate                 226773 non-null datetime64[ns]
id                      226773 non-null int64
guid                    226773 non-null object
link                    226773 non-null object
author                  226773 non-null object
title                   226772 non-null object
description             226772 non-null object
source                  226773 non-null object
user_id                 226773 non-null int64
profile_image_url       226773 non-null object
user_statuses_count     226773 non-null int64
user_friends_count      226773 non-null int64
user_followers_count    226773 non-null int64
user_created_at         226773 non-null object
user_bio                171591 non-null object
user_location           151316 non-null object
lang                    226773 non-null object
code                    226773 non-null object
coords                  144 non-null ob

In [30]:
df_all.drop(['guid','link','title','user_id','profile_image_url','lang','code','coords','source','user_statuses_count','user_friends_count','user_followers_count','user_location'], axis=1, inplace=True)

In [31]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226773 entries, 834 to 20829
Data columns (total 6 columns):
pubdate            226773 non-null datetime64[ns]
id                 226773 non-null int64
author             226773 non-null object
description        226772 non-null object
user_created_at    226773 non-null object
user_bio           171591 non-null object
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 12.1+ MB


In [32]:
df_all.dropna(subset=['description'],inplace=True)

In [33]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226772 entries, 834 to 20829
Data columns (total 6 columns):
pubdate            226772 non-null datetime64[ns]
id                 226772 non-null int64
author             226772 non-null object
description        226772 non-null object
user_created_at    226772 non-null object
user_bio           171591 non-null object
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 12.1+ MB


In [34]:
def preditor(x):
    prediction = clf.predict(vectorizer.transform([clean_tweet(x)]))
    return prediction[0]

In [35]:
start_time = time.time()
y = df_all["description"].apply(preditor)
df_all["predicted"] = y
print(time.time() - start_time)

677.8016431331635


In [36]:
df_all["predicted"].value_counts()

negative    195403
positive    31369 
Name: predicted, dtype: int64

In [37]:
df_all[df_all["predicted"]=="positive"].iloc[0:100]

,pubdate,id,author,description,user_created_at,user_bio,predicted
750,2019-05-01 02:09:21,24750,Vellozo_Huey,@jairbolsonaro mais um toco?\nhttps://t.co/ajVngU1DWl,2008-10-08 18:50:47,🖤Baixista do Huey💀 🌻Pianista do 23:13🖤,positive
293,2019-05-01 02:13:36,24293,BellomoSonia,@MarceloFreixo Meu Deus bolsonaro tbm é ladrão🤣nunca mais votarei nele. Vai se tratar frouxo,2018-05-05 03:01:09,Cristã,positive
229,2019-05-01 02:15:35,24229,marcoreliosoare,"Nessa era Bolsonaro, temos um Brasil depressivo. https://t.co/TuIcTrVqEl",2010-01-14 18:02:35,NaN,positive
128,2019-05-01 02:19:43,24128,ABolsomita,"Bolsonaro falará sobre a Reforma da Previdência no Programa Sílvio Santos, no próximo domingo - https://t.co/UzXeRvJBhu",2016-04-03 20:39:26,"Donde veio o joio? Quer q arranquemos? Não, deixai-os crescer junto c o trigo até a colheita. Depois Arrancai e queimai o joio e recolhei o trigo MT 13:24🇧🇷🇮🇱🕎",positive
112,2019-05-01 02:20:29,24112,psljovemse,"O B R A S I L 🇧🇷 \nestá crescendo, ame-o ou deixe-o.\n\nBrevemente poderemos gritar: ""ACABOU AS REGALIAS E MAMATAS DOS EX-PRESIDENTES"" 😃👏👏👏🙌🏻🙏"".\n\nPresidente Bolsonaro esteve nesta… https://t.co/Yman9dWdMn",2018-03-31 22:04:42,NaN,positive
99,2019-05-01 02:21:31,24099,DonnaF52243669,"Bolsonaro falará sobre a Reforma da Previdência no Programa Sílvio Santos, no próximo domingo - AÍ PESSOAL, Vamos divulgar, ... https://t.co/e2QKUH8HgN",2018-04-13 06:41:24,"🥰🇧🇷🇧🇷🇧🇷🇮🇹🇺🇸🇮🇱😇 PATRIOTA, DIREITA LIBERAL, MÃE, AVÓ, FORMAÇÃO PROFESSORA, AGT. COMERCIO INTERN., TÉCNICA FLORESTAL NORMAS ABNT, GUIADA POR DEUS, EM LUTA BRASIL!!",positive
62,2019-05-01 02:23:21,24062,ninguem_anyone,"@RodrigoMaia Deixe de achar que você, com bem menos votos que Carlos Bolsonaro que é vereador, tem a mesma importância que Jair Bolsonaro, o presidente com 58 milhões de votos, que quase ganhou no primeiro turno. O capitão representa a maioria conservadora e cristã. https://t.co/MouZGZxgYr",2014-07-11 16:22:57,"Conservador. Anti-Globalista. Cristão. Contra a liberação de drogas. Contra o aborto. A favor do armamento civil (posse, porte). Geopolítica estratégica é tudo.",positive
26,2019-05-01 02:24:59,24026,Esqueleto50,"@MarciaBertoline @cardoso_deli @MariseSampaio1 @luizbssantos @PRagazzon @Ivan11473751 @KilkieAlvernaz @denise_A_Sant @HusseinMinho @ValleAmarildo @luanaR8749 @Valqurias4 @betozsantos @RodapeDoPiG @BETOHOMMER @annasil24538637 @eugeniorib1958 @observ1972 @Baro51500518 @diraholanda @HannoucheT @Observo0000 @AnaRoch335 @socialcaipira @ruipalanque @CarlosxuxuDias @SargentoGangrel @Neto821 @Adrihasp @Serena10299551 @gilamarallima @mg_markao @JFerreiraJr2 @MariaNoblatPion @deusakhaly @Laramir41348617 @direitavolver17 @easy_baba @Leo_Xavier_t @HMF58984234 @Heman5791 @EvaldoBonifacio @LalaMD2015 @HeethSchitine @MancioCarla @Denise_A_Santo @chicocuritiba @GGGGGGui80 @emirsader @vnm1418 Esse povo que vota no PT é um povo todo alvoroçado, sem educação! O caba ver no twitter que essa turma do PT fica torcendo contra Bolsonaro e jogando praga pro governo dá errado, isso é uma tristeza! Quem ama o Brasil torce pro governo dá certo!",2017-05-16 14:31:24,"#Bolsonaro2018 Eu sou uma pessoa muito positiva! Devoto de Nossa Senhora! Eu nasci nu, comecei a trabalhar com 6 anos de idade e quero bem a todos!",positive
626,2019-05-01 02:26:14,24626,ninguem_anyone,"@RodrigoMaia Você, com seus ínfimos votos, ninguém sabe como se elegeu. Tu não tens a mesma importância ou autoridade que Jair Bolsonaro, o presidente com 58 milhões de votos que quase ganhou no primeiro turno, tem. O capitão representa a maioria conservadora e cristã. https://t.co/MouZGZxgYr",2014-07-11 16:22:57,"Conservador. Anti-Globalista. Cristão. Contra a liberação de drogas. Contra o aborto. A favor do armamento civil (posse, porte). Geopolítica estratégica é tudo.",positive
574,2019-05-01 02:28:17,24574,RevoIuvionario,"Ontem à noite (30) foi confirmada a presenta do presidente Jair Bolsonaro no Programa Sílvio Santos, que vai ao ar aos domingos, no SBT.\n\nO programa s

In [38]:
df_all[df_all["predicted"]=="negative"].iloc[0:100]

,pubdate,id,author,description,user_created_at,user_bio,predicted
834,2019-05-01 02:06:12,24834,wesleyespindula,"Maia diz que cabe ao Congresso, e não a Bolsonaro, autorizar guerra https://t.co/K2i07XWdGd",2011-06-08 18:12:01,Wesley Espindula Instalaçao Venda Manutençao De Ar Condicionado,negative
832,2019-05-01 02:06:18,24832,diogoandrew85,"@PAULOSP001 @NaitzSprite @caiopilan @Tosilva17 @RaphaelSG14 @Ideias_Radicais @pauloap Isso se você nao tiver vivendo numa Venezuela, queria ver tu falar isso se Haddad fosse presidente com desarmamento e regulação dos meios de comunicação como estava no plano de governo dele... É facil ser liberal surfando na onda conservadora. Agradeça ao Bolsonaro. :)",2016-12-13 16:36:30,Gifted by the earth.,negative
831,2019-05-01 02:06:18,24831,andrearoeirap,"Que surpresa, um lobista da pesca industrial, filho de armadores e aliado do Bolsonaro, puxa, não esperava mesmo https://t.co/Dw51K6vEzl",2017-06-15 18:47:51,"Biólogo e Msc. atleticano, ambientalista, articulista.\nEspecialista em políticas públicas ambientais e financiamento da conservação.\nPerfil pessoal.",negative
829,2019-05-01 02:06:20,24829,_criscorrea,"@RodrigoMaia Mandar nossos homens para Venezuela? Esse negócio só deve ser aprovado se for um exército formado pela família Bolsonaro e seus militantes, fora isso duvido que nossas famílias apóiem essa intervenção ridícula. Não dão conta nem do próprio país e querer se meter no dos outros?",2014-10-08 05:12:42,Criadora da Maenga Toys. 👾 (mãe dos monstros)📍Cristã que às vezes grava vídeos. Blo'Vlogueira um tanto preguiçosa + com análises minuciosas.,negative
827,2019-05-01 02:06:22,24827,AuricelCarvalho,Justiça nega pedido de Flávio Bolsonaro para interromper investigação sobre Queiroz https://t.co/O2Urarfi0e,2010-06-05 14:14:12,Olho por olho dente por dente,negative
826,2019-05-01 02:06:24,24826,MeUniversitario,Bolsonaro vai a banco e na volta encontra protesto na porta de seu condomínio https://t.co/rY0YacbNzO,2015-12-31 21:13:48,NaN,negative
812,2019-05-01 02:06:53,24812,ritajose7,"@UOLNoticias Por falar nisso cadê o Queiroz,Bolsonaro nem toca no assunto.",2017-09-08 01:55:31,NaN,negative
805,2019-05-01 02:07:12,24805,ellychata,@apcify eu toda vez que vejo o bolsonaro:,2019-02-19 11:37:00,noora sætre and nick scratch are my sunshine 🏹🖤,negative
804,2019-05-01 02:07:13,24804,JeanClevis,Filosofia e Sociologia no Governo Bolsonaro! Pq investir menos nesses cursos? https://t.co/VFgylnzecg,2009-03-24 15:10:52,Novo nerd.\nEstuda dez horas por dia mas tem vida social.,negative
803,2019-05-01 02:07:14,24803,followthebail,"Se eu encontrasse um gênio da lâmpada e pudesse fazer 3 pedidos, um deles seria um vídeo do bolsonaro dando o cu.",2011-10-07 12:30:15,JB aqui não,negative


In [39]:
df_positive = df_all[df_all["predicted"]=="positive"]
df_positive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31369 entries, 750 to 20863
Data columns (total 7 columns):
pubdate            31369 non-null datetime64[ns]
id                 31369 non-null int64
author             31369 non-null object
description        31369 non-null object
user_created_at    31369 non-null object
user_bio           21716 non-null object
predicted          31369 non-null object
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 1.9+ MB


In [40]:
df_negative = df_all[df_all["predicted"]=="negative"]
df_negative.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195403 entries, 834 to 20829
Data columns (total 7 columns):
pubdate            195403 non-null datetime64[ns]
id                 195403 non-null int64
author             195403 non-null object
description        195403 non-null object
user_created_at    195403 non-null object
user_bio           149875 non-null object
predicted          195403 non-null object
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 11.9+ MB


In [63]:
# amostra_positivos = []
# for x in range(50):
#      amostra_positivos.append(random.randint(0,len(df_positive)))

In [64]:
# amostra_positivos

In [65]:
# amostra_negativos = []
# for x in range(50):
#      amostra_negativos.append(random.randint(0,len(df_negative)))

In [66]:
# amostra_negativos

In [68]:
amostra_all = []
for x in range(100):
     amostra_all.append(random.randint(0,len(df_all)))

In [69]:
print(amostra_all)

[47383, 212487, 29260, 75602, 105603, 5197, 191758, 192304, 100646, 43245, 226078, 163910, 16502, 114434, 99382, 96907, 216965, 200291, 154, 91640, 154412, 205886, 60979, 143823, 107903, 206337, 79097, 56683, 171415, 193409, 103803, 177489, 145455, 60024, 172732, 93334, 85968, 69221, 193710, 132656, 30505, 120304, 41181, 217686, 90910, 220575, 60448, 143701, 205193, 73260, 93916, 150419, 110700, 26505, 56868, 147919, 152913, 61295, 48638, 51409, 58972, 48659, 64577, 89232, 99798, 216178, 22485, 151274, 49425, 202065, 106653, 186676, 63783, 202440, 201120, 179858, 195694, 172403, 197554, 76002, 154271, 153504, 72254, 5616, 33181, 173468, 20283, 204479, 186358, 207557, 27473, 197568, 88084, 149128, 17470, 37314, 113288, 560, 93505, 66105]


In [70]:
df_all.iloc[amostra_all][0:10]["description"]

37824    Falta prosseguir com a do Bolsonaro que vários robôs se vestiram de caixa 2 e tem o véio da Havan no meio https://t.co/xzZKIVI3ys                                                                                                                                                                                                    
63040    Por apoiar Bolsonaro, Gabriel Diniz tem morte comemorada https://t.co/KcdrTQAqVR                                                                                                                                                                                                                                                     
61389    @opropriolavo Taí o lixo q o Bolsonaro e o filhos idolatram\n@GeneralMourao \n@Gen_VillasBoas \n@GeneralGirao \n@GalTheophilo \n@Gen_Ex_Freitas                                                                                                                                                                                  

In [71]:
df_all.iloc[amostra_all][10:20]["description"]

23851    #WelcomeToBrazilBts\nBOLSONARO ACABOU PRA VC PORQUE HOSEOK E NAMJOON ESTÃO CHEGADA P TIRAR TEU LUGAR NA PRESIDÊNCIA DO BRASIL HOPE                                                                                                                                                                            
34383    @JFMargarida Já tem uma onda de negacio aparecendo os primeiros negacionista do aquecimento global no país: ele, Eduardo Bolsonaro e por aí vai....                                                                                                                                                           
5715     @CarlosAGoulart1 @Contreiras2016 Olhaaa, vendo que o Congresso, o STF, e a meanstream mídia não quer consertar o país e só atrapalhar pra não perderem suas regalias ao custo da bancarrota do país, de que outra forma Bolsonaro vai conseguir fazer o que precisa os que ele faça?                          
83638    @LocoBaltar @PauloMartins10 vai voltar para o ministéri

In [79]:
df_all.iloc[amostra_all][20:30]["description"]

23851    #WelcomeToBrazilBts\nBOLSONARO ACABOU PRA VC PORQUE HOSEOK E NAMJOON ESTÃO CHEGADA P TIRAR TEU LUGAR NA PRESIDÊNCIA DO BRASIL HOPE                                                                                                                                                                            
34383    @JFMargarida Já tem uma onda de negacio aparecendo os primeiros negacionista do aquecimento global no país: ele, Eduardo Bolsonaro e por aí vai....                                                                                                                                                           
5715     @CarlosAGoulart1 @Contreiras2016 Olhaaa, vendo que o Congresso, o STF, e a meanstream mídia não quer consertar o país e só atrapalhar pra não perderem suas regalias ao custo da bancarrota do país, de que outra forma Bolsonaro vai conseguir fazer o que precisa os que ele faça?                          
83638    @LocoBaltar @PauloMartins10 vai voltar para o ministéri

In [72]:
df_all.iloc[amostra_all][30:40]["description"]

6558     Ei @RodrigoMaia, você tá querendo escantear Bolsonaro e tocar o governo como se fosse um parlamentarismo? Por que não faz um plebiscito?                                                                                                                                                           
25104    Gleisi vê fiasco na manifestação pró-Bolsonaro https://t.co/vUzYMninJa                                                                                                                                                                                                                             
90048    @JanainaDoBrasil No meio acadêmico vi professor me chamar de bolsominion de merda e me mandar sair da academia, que lá não era meu lugar. Arrisquei minha graduação indo a rua e manifestando meu apoio irrestrito a Bolsonaro. Se ele não ganhasse nem sei o que seria de mim lá dentro. Arrisquei
838      @exame Só o fato de Bolsonaro ter vencido Haddad trouxe esperança ao brasileiro...      

In [61]:
df_positive.iloc[amostra_positivos][40:50]["description"]

59180    @jairbolsonaro BOM DIA MEU PRESIDENTE BOLSONARO                                                                                                                                                                                                                                                           
30480    Não haverá mais "BOM DIA", se o Governo Bolsonaro não tiver a sua PRESENÇA no dia 26 / 05.                                                                                                                                                                                                                
57124    NÃO PASSARÁ NA GLOBO: Bolsonaro recebe HOMENAGEM EMOCIONANTE em Recife https://t.co/UqPq3cEKzl via @YouTube                                                                                                                                                                                               
13219    O deputado Paulinho da Força afirmou, durante o ato do 1º de Maio d

In [74]:
df_all.iloc[amostra_all][50:60]["description"]

58672    Como sou servo de Cristo, não uso palavras como "idiota", "imbecil", "otário", "burro", porque são pejorativas. Prefiro usar "néscio", "tolo", "estulto" e "insensato", que são as palavras usadas na Bíblia. Todavia, concordo com Bolsonaro. #BolsonaroTemRazao                                      
10606    @dw_brasil Já pegou fogo, não posso fazer nada.\n\nBolsonaro, Jair                                                                                                                                                                                                                                     
35443    É muito dinheiro indo pelo ralo, tem que intervir sim, a BR é uma estatal (Pública), Quantos ralos ainda existem? A quem interessa? \n#SabadoLigaDaJustiçaSDV\n#SabadoDetremuraSDV\n\n"Bolsonaro diz que vai tentar rescindir contrato da Petrobras com a McLaren"  https://t.co/Gp1mmvQg8d            
50365    5 meses de governo Bolsonaro minha mãe continua o discurso que agora vai aca

In [75]:
df_all.iloc[amostra_all][60:70]["description"]

95604    @JornalDaCidadeO Mais o povo voto a favor da armas e não vai ouvido, e a Vitória de Bolsonaro já foi um claro plebiscito!                                                                                                                                                                                        
44973    A verdade é que, para o bem ou o mal, o Bolsonaro adiantou uma discussão que já deveria existir há anos: até quando vai durar a UFMT?\n\nA UFMT tem um orçamento maior que Várzea Grande, para se ter ideia.\nOrçamento que é destinado, e ainda assim, gera prejuízo.                                           
19932    @frcabreira @Eliana44717913 @jc_pe Pernambuco diz fora Bolsonaro                                                                                                                                                                                                                                                 
33890    O problema do Bolsonaro não é ser quinta-série

In [76]:
df_all.iloc[amostra_all][70:80]["description"]

18717    @XSplatterhouse @bellamoura18 @jairbolsonaro @RudyGiuliani @MajorVitorHugo Ele no mínimo consegue governar, bolsonaro só fica postando bobagem no twitter e viajando sem ser convidado, metade disso nem é ele quem faz.                                                                                        
62884    @Jujuca1987 Modo Bolsonaro de resolver problemas! Culpe quem afere a audiência, popularidade ou desemprego!                                                                                                                                                                                                     
16455    @jairbolsonaro Bolsonaro, somos 30 mil Aprovados no maior Concurso da CEF 2014!\nA Caixa precisa de nós!\nAté hoje, nos governos anteriores, nada foi feito!\nConfiamos em Você!\nNão nos decepcione!\nVamos oxigenar essa grandiosa Empresa!\nFaremos história juntos!\nEstamos prontos!\nVamos!\n\n#ConvocaCEF
21576    me da um ranço da minha mãe defendendo o bolsonar

In [77]:
df_all.iloc[amostra_all][80:90]["description"]

24286    @MiriamLeitaoCom Se sua visão de governo é mesma dos últimos presidentes de esquerda de viés socialista você está certa, ele realmente não sabe governar. Más pra nós brasileiros de direita e conservadores Bolsonaro está fazendo um governo incrível, sem negociatas, conlúio, conchavo, enfim show.
21872    Bolsonaro é burro e governa como se estivesse em um churrasco, diz Pondé\n\nFilósofo de direita critica o governo de Jair Bolsonaro e acredita que o presidente precisa mudar o tom — se não, um novo impeachment pode acontecer\n https://t.co/GCgHS8i6XF                                             
54511    #13deMaio\nPra quer segurança pro trabalhador né Bolsonaro ?\n\nJair Bolsonaro anuncia redução de 90% de normas de segurança no trabalho\nhttps://t.co/aljOaW7PZr                                                                                                                                      
27281    @jnascim @opropriolavo todo dia o Bolsonaro fazendo algo estúpido pra enalte

In [78]:
df_all.iloc[amostra_all][90:100]["description"]

54098    Confira a matéria completa. https://t.co/a5YXGL4yOK                                                                                                                                                                                                                              
5301     Eu votei no Bolsonaro para ele chamar de idiota o idiota. \nSe fosse para chamar o idiota de intelectual eu teria votado no PT! https://t.co/K2R0b4B5WA                                                                                                                          
26972    Vi até quem votou em Bolsonaro manifestando nas ruas hoje                                                                                                                                                                                                                        
7040     "Eu não sei se vou me aposentar por que o Bolsonaro tá tirando tudo" \n\n"Eu votei nesse homem e me arrependi"\n\n"As energias negativas influ

In [ ]:
writer = pd.ExcelWriter(r'data/12_positive.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df_positive.to_excel(writer)
writer.close()

In [ ]:
writer = pd.ExcelWriter(r'data/12_negative.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df_negative.to_excel(writer)
writer.close()